In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import matplotlib.pyplot as plt
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train = pd.read_csv('../input/fossl-zindi/Train.csv')
test = pd.read_csv('../input/fossl-zindi/Test.csv')
ss = pd.read_csv('../input/fossl-zindi/SampleSubmission.csv')
dict_s = pd.read_csv('../input/fossl-zindi/DataDictionary.csv')

In [50]:
# train.groupby('sku_name').get_group('ARLENARDLAKE')[sellout]/1013
# test.loc[test.sku_name.isin(train.sku_name)]
# plt.plot(train.groupby(['year','month']).agg('sum')[sellin[1]].values)
# plt.plot(train.groupby(['year','month']).agg('sum')[sellin[2]].values/1013)
# plt.plot(train.groupby(['year','month']).agg('sum')[sellout[1:]].sum(axis = 1).values)
# plt.plot(train.groupby(['year','month']).agg('sum')[sellin[1:]].sum(axis = 1).values)
# plt.plot(train.groupby(['year','month']).agg('sum')[onhand[1:]].sum(axis = 1).values)
# plt.plot(train.groupby(['year','month']).agg('sum')[onhand[2]].values/1013)
# plt.plot(train.groupby(['year','month']).agg('sum')[sellin[3]].values)
# plt.plot(train.groupby(['year','month']).agg('sum')[sellout[3]].values)
# plt.plot(train.groupby(['year','month']).agg('sum')[sellin[6]].values)
target = 'sellin'
prediction_columns = test.columns.tolist()
encoding_columns = [t for t in train.columns if t not in prediction_columns]

In [52]:
# encoding_columns
new_id_test = [idd for idd in test.sku_name.unique().tolist() if idd not in train.sku_name.unique().tolist()]
# encoding_columns
new_id_train = [idd for idd in train.sku_name.unique().tolist() if idd not in test.sku_name.unique().tolist()]

In [55]:
len(new_id_train)

In [4]:
# sum((train.sellout_channel_2.unique()/1013)%1)
train.sellout_channel_1.min()

In [17]:
sellin = ['sellin','sellin_channel_1','sellin_channel_2','sellin_channel_3','sellin_channel_4',
         'sellin_channel_5','sellin_channel_6','sellin_channel_7','sellin_channel_8']
sellout = ['sellout','sellout_channel_1','sellout_channel_2','sellout_channel_3','sellout_channel_4',
         'sellout_channel_5','sellout_channel_6','sellout_channel_7','sellout_channel_8','sellout_channel_9',
           'sellout_channel_10']
onhand= ['onhand_inventory','onhand_inventory_channel_1','onhand_inventory_channel_2','onhand_inventory_channel_3','onhand_inventory_channel_4',
         'onhand_inventory_channel_5','onhand_inventory_channel_6','onhand_inventory_channel_7','onhand_inventory_channel_8','onhand_inventory_channel_9',
           'onhand_inventory_channel_10']

In [30]:
dict_s.loc[dict_s['Column Name'] == 'sellin','Description']

In [7]:
train.groupby('sku_name').get_group(train.sku_name.unique()[0])

In [56]:
df_train = train.copy()
df_test = test.copy()
df_train[sellin+sellout+onhand] = df_train[sellin+sellout+onhand]/1013

In [73]:
sku_name_target_mean = df_train.groupby('sku_name')['sellin'].mean() 
df_test['sku_name_target_enc'] = df_test['sku_name'].map(sku_name_target_mean)
df_test['sku_name_target_enc'].fillna((df_test['sku_name_target_enc'].mean()), inplace=True)

In [78]:
from sklearn.model_selection import KFold
tmp = np.repeat(np.nan, df_train.shape[0])
kf = KFold(n_splits = 5, shuffle=True,random_state = 19910325)
for idx_1, idx_2 in kf.split(df_train):
    target_mean = df_train.iloc[idx_1].groupby('sku_name')['sellin'].mean()

    tmp[idx_2] = df_train['sku_name'].iloc[idx_2].map(target_mean)
df_train['sku_name_target_enc'] = tmp

In [79]:
import lightgbm as lgbm# .loc[df_test.sku_name == new_id_test[0]]
from sklearn.preprocessing import LabelEncoder as LE

In [82]:
all_data = pd.concat([df_train[prediction_columns],df_test[prediction_columns]])

In [86]:
le = LE()
all_data['enc_sku'] = le.fit_transform(all_data['sku_name'])
df_train['enc_sku'] = all_data.iloc[:df_train.shape[0]]['enc_sku']
df_test['enc_sku'] = all_data.iloc[df_train.shape[0]:]['enc_sku']

# df_test['enc_sku'] = le.transform(df_test['sku_name'])

In [87]:
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))

def feval_RMSPE(preds, lgbm_train):
    labels = lgbm_train.get_label()
    return 'RMSPE', round(rmspe(y_true = labels, y_pred = preds),5), False

params = {
      "objective": "rmse", 
      "metric": "rmse", 
      "boosting_type": "gbdt",
      'early_stopping_rounds': 30,
      'learning_rate': 0.01,
#       'lambda_l1': 1,
#       'lambda_l2': 1,
      'feature_fraction': 0.8,
      'bagging_fraction': 0.8,
  }

In [88]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3, random_state=19901028, shuffle=True)
oof = pd.DataFrame()                 # out-of-fold result
models = []                          # models
scores = 0.0                         # validation score

In [93]:
target = 'sellin'
prediction_columns = df_test.columns.tolist()
encoding_columns = [t for t in df_train.columns if t not in prediction_columns]
X = df_train.drop(encoding_columns+[target,'sku_name'],axis=1)
y = df_train[target]

In [94]:
X

In [135]:
%%time
for fold, (trn_idx, val_idx) in enumerate(kf.split(X, y)):

    print("Fold :", fold+1)
    
    # create dataset
    X_train, y_train = X.loc[trn_idx], y[trn_idx]
    X_valid, y_valid = X.loc[val_idx], y[val_idx]
    
    #RMSPE weight
    weights = 1/np.square(y_train+1)
    lgbm_train = lgbm.Dataset(X_train,y_train,weight = weights)

    weights = 1/np.square(y_valid+1)
    lgbm_valid = lgbm.Dataset(X_valid,y_valid,reference = lgbm_train,weight = weights)
    
    # model 
    model = lgbm.train(params=params,
                      train_set=lgbm_train,
                      valid_sets=[lgbm_train, lgbm_valid],
                      num_boost_round=5000,         
                      feval=feval_RMSPE,
                      verbose_eval=100,
                      categorical_feature = ['enc_sku']                
                     )
    
    # validation 
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)

    RMSPE = round(rmspe(y_true = y_valid, y_pred = y_pred),0)
    print(f'Performance of the　prediction: , RMSPE: {RMSPE}')

    #keep scores and models
    scores += RMSPE / 3
    models.append(model)
    print("*" * 100)

In [136]:
X_test = df_test[X_train.columns.tolist()].copy()

In [137]:
df_test['Item_ID'] = df_test['sku_name'] +'_'+ df_test['month'].astype(str)+ '_'+df_test['year'].astype(str)
y_pred = df_test[['Item_ID']]
target = np.zeros(len(X_test))

#light gbm models
for model in models:
    pred = model.predict(X_test[X_valid.columns], num_iteration=model.best_iteration)
    target += pred / len(models)
y_pred = y_pred.assign(target_y= target)
sub = ss.merge(y_pred,on = 'Item_ID')
sub['target'] = sub['target_y']*1013
sub['target'] = sub['target'].astype('int') 
sub = sub[['Item_ID','target']]

In [138]:
sub

In [133]:
sub.to_csv('submission.csv',index = False)

In [126]:
# df_test.apply(lambda x: '_'.join(x['sku_name'],x['month']))
